In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
# Mapping for French month names to numbers
month_mapping = {
    'janvier': '01',
    'février': '02',
    'mars': '03',
    'avril': '04',
    'mai': '05',
    'juin': '06',
    'juillet': '07',
    'août': '08',
    'septembre': '09',
    'octobre': '10',
    'novembre': '11',
    'décembre': '12'
}

def parse_release_date(date_str):
    # Converts "12 mars 2025" into "2025-03-12"
    if pd.isna(date_str):
        return None
    parts = date_str.strip().split()
    if len(parts) != 3:
        return date_str
    day, month_fr, year = parts
    month = month_mapping.get(month_fr.lower(), '01')
    return f"{year}-{month}-{day.zfill(2)}"

In [3]:
def duration_to_minutes(duration_str):
    # Converts "1h 26min" into integer minutes (86)
    if pd.isna(duration_str):
        return None
    match = re.search(r'(\d+)\s*h\s*(\d+)?', duration_str)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2)) if match.group(2) else 0
        return hours * 60 + minutes
    return None

In [4]:
def extract_number(text):
    # Remove any spaces and then check if the remaining text is a number
    if pd.isna(text):
        return None
    cleaned = text.replace(" ", "")
    try:
        return int(cleaned)
    except ValueError:
        # Fallback: extract the first sequence of digits if conversion fails
        import re
        match = re.search(r'(\d+)', cleaned)
        return int(match.group(1)) if match else None

In [5]:
def split_list(text, separator=','):
    # Splits a string into a trimmed list
    if pd.isna(text) or text.strip() == "":
        return []
    return [x.strip() for x in text.split(separator) if x.strip()]

In [6]:
def clean_awards(text):
    # Separates awards and nominations from a string like "1 prix et 1 nomination" or "20 nominations"
    awards = None
    nominations = None
    if pd.isna(text) or text.strip() == "":
        return awards, nominations
    match_awards = re.search(r'(\d+)\s*prix', text, re.IGNORECASE)
    match_nominations = re.search(r'(\d+)\s*nomination', text, re.IGNORECASE)
    if match_awards:
        awards = int(match_awards.group(1))
    if match_nominations:
        nominations = int(match_nominations.group(1))
    return awards, nominations

In [7]:
def clean_viewer_critics(text):
    # Splits a string like "566 notes, 241 critiques" into a tuple (notes, critiques)
    notes = None
    critiques = None
    if pd.isna(text) or text.strip()=="":
        return notes, critiques
    match_notes = re.search(r'(\d+)\s*notes', text, re.IGNORECASE)
    match_critiques = re.search(r'(\d+)\s*critiques', text, re.IGNORECASE)
    if match_notes:
        notes = int(match_notes.group(1))
    if match_critiques:
        critiques = int(match_critiques.group(1))
    return notes, critiques

In [8]:
def parse_week_period(period_text):
    """
    Given a French week period string like "05 au 12 mars 2025",
    extract start and end dates and return a tuple:
      (formatted_period, iso_week_number)
    where formatted_period is "dd/mm/yyyy – dd/mm/yyyy" using the same month/year for both dates.
    """
    if pd.isna(period_text) or period_text.strip() == "":
        return None, None
    # Pattern expecting something like: "05 au 12 mars 2025"
    pattern = r'(\d{1,2})\s*au\s*(\d{1,2})\s*(\w+)\s*(\d{4})'
    match = re.search(pattern, period_text.strip(), re.IGNORECASE)
    if not match:
        return period_text, None
    start_day, end_day, month_fr, year = match.groups()
    month = month_mapping.get(month_fr.lower(), '01')
    start_date_str = f"{start_day.zfill(2)}/{month}/{year}"
    end_date_str = f"{end_day.zfill(2)}/{month}/{year}"
    # Determine ISO week number using the start date
    try:
        dt_start = datetime.strptime(f"{year}-{month}-{start_day.zfill(2)}", "%Y-%m-%d")
        iso_week = dt_start.isocalendar()[1]
    except Exception:
        iso_week = None
    formatted_period = f"{start_date_str} – {end_date_str}"
    return formatted_period, iso_week

In [9]:
def get_season(date_str):
    """
    Given a date string in "YYYY-MM-DD", returns the season (Winter, Spring, Summer, Autumn).
    """
    if pd.isna(date_str):
        return None
    try:
        dt = datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        return None
    month = dt.month
    # Northern hemisphere seasons
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Autumn"

In [11]:
def clean_data(df):
    # film_title: remove extra spaces
    df['film_title'] = df['film_title'].str.strip()
    
    # film_url: extract film_id
    df['film_id'] = df['film_url'].apply(
        lambda x: re.search(r'fichefilm_gen_cfilm=(\d+)', x).group(1) if re.search(r'fichefilm_gen_cfilm=(\d+)', x) else None
    )
    
    # release_date: reformat date
    df['release_date'] = df['release_date'].apply(parse_release_date)
    
    # Derive season from release_date
    df['release_season'] = df['release_date'].apply(get_season)
    
    # duration: convert to minutes
    df['duration_minutes'] = df['duration'].apply(duration_to_minutes)
    
    # age_classification: remove extra spaces
    df['age_classification'] = df['age_classification'].str.strip()
    
    # producers: split and expand into columns
    producers_split = df['producers'].apply(lambda x: split_list(x))
    df['producers_count'] = producers_split.apply(len)
    max_producers = producers_split.apply(len).max()
    for i in range(max_producers):
        df[f'producer_{i+1}'] = producers_split.apply(lambda x: x[i] if len(x) > i else None)
    
    # director: clean spaces
    df['director'] = df['director'].str.strip()
    
    # top_stars: split and expand into columns
    top_stars_split = df['top_stars'].apply(lambda x: split_list(x))
    df['top_stars_count'] = top_stars_split.apply(len)
    max_stars = top_stars_split.apply(len).max()
    for i in range(max_stars):
        df[f'top_star_{i+1}'] = top_stars_split.apply(lambda x: x[i] if len(x) > i else None)
    
    # languages: count number of languages instead of list
    df['language_count'] = df['languages'].apply(lambda x: len(split_list(x, separator=",")))
    
    # film_nationality: count number of nationalities
    df['film_nationality_count'] = df['film_nationality'].apply(lambda x: len(split_list(x, separator=",")))
    
    # filming_secrets: extract number and cast to int (if missing, set to 0)
    df['filming_secrets_num'] = df['filming_secrets'].apply(lambda x: extract_number(x))
    df['filming_secrets_num'] = df['filming_secrets_num'].apply(lambda x: int(x) if pd.notna(x) else 0)
    
    # Process week periods for fr_entry_week and us_entry_week
    for col in ['fr_entry_week', 'us_entry_week']:
        period, iso_week = zip(*df[col].apply(lambda x: parse_week_period(x)))
        df[f'{col}_period'] = period
        df[f'{col}_iso_week'] = [int(i) if pd.notna(i) else 0 for i in iso_week]
    
    # Process fr_entries and us_entries as numbers (remove spaces and cast to int)
    for col in ['fr_entries', 'us_entries']:
        df[f'{col}_num'] = df[col].apply(lambda x: extract_number(x))
        df[f'{col}_num'] = df[f'{col}_num'].apply(lambda x: int(x) if pd.notna(x) else 0)
    
    # awards: separate awards and nominations and cast to int (defaulting to 0)
    awards_data = df['awards'].apply(clean_awards)
    df['award_count'] = awards_data.apply(lambda x: int(x[0]) if pd.notna(x[0]) else 0)
    df['nomination_count'] = awards_data.apply(lambda x: int(x[1]) if pd.notna(x[1]) else 0)
    # Create total awards+nomination column, as int
    df['total_awards_nomination'] = df.apply(lambda row: int(row['award_count']) + int(row['nomination_count']), axis=1)
    
    # associated_genres: keep only count (drop the list version)
    df['associated_genres_count'] = df['associated_genres'].apply(lambda x: len(split_list(x, separator=",")))
    
    # press_critics_count: extract number and cast to int
    df['press_critics_count_num'] = df['press_critics_count'].apply(lambda x: extract_number(x))
    df['press_critics_count_num'] = df['press_critics_count_num'].apply(lambda x: int(x) if pd.notna(x) else 0)
    
    # viewer_critics_count: split into notes and critiques and cast them to int
    viewer_data = df['viewer_critics_count'].apply(clean_viewer_critics)
    df['viewer_notes'] = viewer_data.apply(lambda x: int(x[0]) if pd.notna(x[0]) else 0)
    df['viewer_critiques'] = viewer_data.apply(lambda x: int(x[1]) if pd.notna(x[1]) else 0)
    
    # synopsis: add a column for the length of the synopsis
    df['synopsis_length'] = df['synopsis'].apply(lambda x: len(x) if pd.notna(x) else 0)
    
    return df

if __name__ == '__main__':
    input_path = '/Users/michaeladebayo/Documents/Simplon/brief_projects/movie_prediction/scraping/allocinescraper/films.csv'
    output_path = '/Users/michaeladebayo/Documents/Simplon/brief_projects/movie_prediction/scraping/allocinescraper/films_cleaned.csv'
    
    df = pd.read_csv(input_path)
    df_cleaned = clean_data(df)
    df_cleaned.to_csv(output_path, index=False)
    print(f"Cleaned data saved to {output_path}")

Cleaned data saved to /Users/michaeladebayo/Documents/Simplon/brief_projects/movie_prediction/scraping/allocinescraper/films_cleaned.csv


In [12]:
pd.set_option('display.max_columns', None)

df_cleaned.head()

,film_title,film_url,film_image_url,release_date,duration,age_classification,producers,director,top_stars,press_rating,viewer_rating,languages,distributor,year_of_production,film_nationality,filming_secrets,fr_entry_week,us_entry_week,fr_entries,us_entries,awards,budget,associated_genres,press_critics_count,viewer_critics_count,synopsis,film_id,release_season,duration_minutes,producers_count,producer_1,producer_2,producer_3,producer_4,top_stars_count,top_star_1,top_star_2,top_star_3,language_count,film_nationality_count,filming_secrets_num,fr_entry_week_period,fr_entry_week_iso_week,us_entry_week_period,us_entry_week_iso_week,fr_entries_num,us_entries_num,award_count,nomination_count,total_awards_nomination,associated_genres_count,press_critics_count_num,viewer_notes,viewer_critiques,synopsis_length
0,Délocalisés,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://fr.web.img6.acsta.net/c_310_420/img/c7...,2025-03-12,1h 26min,Tout public,"Ali Boughéraba,Redouane Bougheraba,Germain Blo...",Ali Boughéraba,"Redouane Bougheraba,Vanessa Guide,Ahassan Uddin",2.8,2.5,Français,Studiocanal / TF1 Studio,2025,NaN,8 anecdotes,NaN,NaN,NaN,NaN,NaN,-,Comédie,4 critiques,"566 notes, 241 critiques",Le jour où Redouane va obtenir sa promotion et...,1000001269,Spring,86,4,Ali Boughéraba,Redouane Bougheraba,Germain Blot,Cédric Dosne,3,Redouane Bougheraba,Vanessa Guide,Ahassan Uddin,1,0,8,None,0,None,0,0,0,0,0,0,1,4,566,241,717
1,À bicyclette !,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://fr.web.img2.acsta.net/c_310_420/img/9c...,2025-02-26,1h 29min,Tout public,"Mathias Mlekuz,Philippe Rebbot",Mathias Mlekuz,"Mathias Mlekuz,Philippe Rebbot,Josef Mlekuz",3.8,4.1,Français,Ad Vitam,2024,NaN,12 anecdotes,05 au 12 mars 2025,NaN,108 029,NaN,1 prix et 1 nomination,-,Comédie dramatique,24 critiques,"2612 notes, 715 critiques","De l’Atlantique à la mer Noire, Mathias embarq...",323031,Winter,89,2,Mathias Mlekuz,Philippe Rebbot,None,None,3,Mathias Mlekuz,Philippe Rebbot,Josef Mlekuz,1,0,12,05/03/2025 – 12/03/2025,10,None,0,108029,0,1,1,2,1,24,2612,715,280
2,Black Dog,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://fr.web.img5.acsta.net/c_310_420/img/99...,2025-03-05,1h 50min,Tout public,"Hu Guan,Rui Ge",Hu Guan,"Eddie Peng,Liya Tong,Jia Zhangke",4.2,4.1,Chinois,Memento,2024,NaN,11 anecdotes,05 au 12 mars 2025,NaN,62 059,NaN,NaN,-,Drame,30 critiques,"1638 notes, 165 critiques",Lang revient dans sa ville natale aux portes d...,1000000974,Spring,110,2,Hu Guan,Rui Ge,None,None,3,Eddie Peng,Liya Tong,Jia Zhangke,1,0,11,05/03/2025 – 12/03/2025,10,None,0,62059,0,0,0,0,1,30,1638,165,283
3,L’Attachement,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://fr.web.img3.acsta.net/c_310_420/img/83...,2025-02-19,1h 45min,Tout public,"Carine Tardieu,Raphaële Moussafir",Carine Tardieu,"Valeria Bruni Tedeschi,Pio Marmaï,Vimala Pons",4.0,4.2,Français,Diaphana Distribution,2024,"France,Belgique",7 anecdotes,05 au 12 mars 2025,NaN,128 314,NaN,NaN,-,Drame,34 critiques,"3156 notes, 383 critiques","Sandra, quinquagénaire farouchement indépendan...",313781,Winter,105,2,Carine Tardieu,Raphaële Moussafir,None,None,3,Valeria Bruni Tedeschi,Pio Marmaï,Vimala Pons,1,2,7,05/03/2025 – 12/03/2025,10,None,0,128314,0,0,0,0,1,34,3156,383,219
4,The Brutalist,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://fr.web.img5.acsta.net/c_310_420/img/22...,2025-02-12,3h 34min,Tout public avec avertissement,"Brady Corbet,Mona Fastvold",Brady Corbet,"Adrien Brody,Felicity Jones,Guy Pearce",4.3,4.0,"Anglais, Hongrois",Universal Pictures International France,2024,"Grande-Bretagne,U.S.A.,Hongrie",13 anecdotes,12 au 19 février 2025,24 au 27 janvier 2025,142 318,2 873 057,11 prix et 17 nominations,-,Drame,42 critiques,"3985 notes, 488 critiques","Fuyant l’Europe d’après-guerre, l’architecte v...",269838,Winter,214,2,Brady Corbet,Mona Fastvold,None,None,3,Adrien Brody,Felicity Jones,Guy Pearce,2,3,13,12/02/2025 – 19/02/2025,7,24/01/2025 – 27/01/2025,4,142318,2873057,1